In [ ]:

import gym
import numpy as np
%pip install rl_visualizer
from rl_visualizer import RLVisualizer
%pip install stable_baselines3
%pip install "gym[box2d]" 
from stable_baselines3 import PPO

def train_and_visualize(env_name="LunarLander-v2", total_timesteps=1000, visualize=True):
    # Create the environment
    env = gym.make(env_name)

    # Initialize the model
    model = PPO("MlpPolicy", env, verbose=1)

    # Train the model

    model.learn(total_timesteps=total_timesteps)

    # Visualization configuration
    config = {
        "history_length": 200,
        "pause_time": 0.001,
        "show_entropy": True,
        "show_action_counts": True,
        "show_rewards": True,
        "colors": ["gray"],
        "highlight_color": "blue",
        "figsize": (10, 10),
    }
    env = gym.make(env_name, render_mode="human")
    # Initialize the visualizer
    visualizer = RLVisualizer(env.action_space, config)

    # Test the model with visualization
    obs, _ = env.reset()
    visualizer.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)

        # Obtain the action probabilities from the model's policy
        obs_tensor = model.policy.obs_to_tensor(obs)[0]
        dis = model.policy.get_distribution(obs_tensor)
        probs = dis.distribution.probs.cpu().detach().numpy()

        # Update visualization with current action probabilities and reward
        visualizer.update_visualization(probs, reward=reward, chosen_action=action)
        env.render()
if __name__ == "__main__":
    train_and_visualize()
